In [51]:
import os
from glob import glob
import warnings
import random
warnings.filterwarnings('ignore')
import datetime

import import_ipynb
import abuse_detecting_preprocessing as pp
import abuse_detecting_databuild as b

import pandas as pd
import numpy as np

import fasttext

import tensorflow as tf

_1DCNN_model = tf.keras.models.load_model('1DCNN_model.h5')
BiLSTM_model = tf.keras.models.load_model('BiLSTM_model.h5')
GRU_model = tf.keras.models.load_model('GRU_model.h5')
fasttext_model = fasttext.load_model('/Users/kim-yongjun/Documents/abuse_detecting/fasttext_model.bin')


In [2]:
def erase_bot(data):

    # 첫 번째 열의 값이 '@'로 시작하는 행을 필터링하여 봇으로 간주하고 제거
    idxs = data[data.str.startswith('@') == True].index
    data.loc[idxs] = None
    bot_cnt = len(idxs)
    print('data.shape : ', data.shape, 'None 처리한 봇 수 : ', bot_cnt, '\n')
    return data
# erase_bot 함수 사용 예시
# data = erase_bot(data)


In [3]:
def null_finder(data): # 공백만 있는 메세지 제거
    print('공백 제거 전\n', data.isnull().value_counts())
    data = data.dropna()
    print('공백 제거 후 \n', data.isnull().value_counts())
    
    return data

In [41]:
def erase_emoticons(data):
# 이모티콘들이 대부분 ::smiling::으로 표시되기에 이 부분을 제거하고 이어 붙이는 작업을 수행.
# ex) ::smiling:: 페이커 잘한다 > 페이커 잘한다 
    cnt = 0
    try:
        cnt = 0
        for i in range(len(data)):
            if ':' in data[i]:
                l = data[i].find(':')
                r = data[i].rfind(':')
                data[i] = data[i][:l] + data[i][r+1:]
                cnt += 1
    except Exception as e:
        print("An error occurred:", e)
    return data

In [42]:
def soft_ensemble(models, data): 
# 욕설 0 , 정상 1 / 세 모델 결과의 평균이 0.3 미만일 경우(확실한 경우)를 0 으로 라벨링
# Pseudo labeled data를 만들기 위함.
    pred = np.column_stack([m.predict(data) for m in models])
    pred = np.mean(pred, axis = 1)
    pred = np.array([0 if p <= 0.3 else 1 for p in pred])
    return pred
    

In [44]:
def makesample(size):
# 전체 데이터에서 size만큼의 크기의 데이터 추출
    data = pd.DataFrame()
    r = glob(os.getcwd() + '/chatting_datas/*.csv')
    for i in r:
        data = pd.concat([data, pd.read_csv(i)], ignore_index = True)
        
    df = data['message']
    message = erase_bot(df)
    message = null_finder(message)
    message = erase_emoticons(message)
    message = message.sample(size)
    
    vectors = b.making_x_train(message, embedding_length, fasttext_model, n_gram)
    return message, vectors

In [46]:
def activation_learning_data(models, size):
# 세 모델의 결과값 평균치가 0.3 이상 0.7 미만인 데이터들만 추출 (욕설인지 아닌지 애매한 데이터들)
    m, vec = makesample(size)
    pred = np.column_stack([m.predict(vec) for m in models])
    pred = np.mean(pred, axis = 1)
    pred = np.array([True if 0.3 <= p <= 0.7 else False for p in pred])
    m = pd.DataFrame(m)
    m['label'] = pred
    target = m[m['label'] == True]
    return target[['message']]

In [62]:
models = [_1DCNN_model, BiLSTM_model, GRU_model]
embedding_length = 40
n_gram = 5
size = 10000
today = datetime.datetime.today().strftime("%Y_%m_%d_%H_%M") 
messages, vec = makesample(size)
messages = pd.DataFrame(messages)

data.shape :  (1007079,) None 처리한 봇 수 :  9367 

공백 제거 전
 message
False    997694
True       9385
Name: count, dtype: int64
공백 제거 후 
 message
False    997694
Name: count, dtype: int64
An error occurred: 63


In [63]:
today

'2024_05_21_17_56'

In [64]:
pseudo_labeled_data = soft_ensemble(models, vec)
messages['label'] = pseudo_labeled_data
messages.to_csv(os.getcwd() + f'/chatting_datas/labeled_datas/{today}_pseudo.csv')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


In [65]:
act_data = activation_learning_data(models, 10000)
act_data.to_csv(os.getcwd() + f'/chatting_datas/{today}_act_learning.csv')

data.shape :  (1007079,) None 처리한 봇 수 :  9367 

공백 제거 전
 message
False    997694
True       9385
Name: count, dtype: int64
공백 제거 후 
 message
False    997694
Name: count, dtype: int64
An error occurred: 63
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
